# Data Segmentation

- Preprocessing

In [3]:
# Loading and cleaning
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Frank\UoT 2024-2025\MIE8888 Project\M18\RUIWU') 
from helper_functions import * # Import Ruiwu's helper functions for data cleaning, slicing and aggregating

full_dataset = r"C:\Frank\UoT 2024-2025\MIE8888 Project\full_dataset_2.11.csv"
df = pd.read_csv(full_dataset, encoding="Windows-1252")

# Define trash columns
trash_columns = ['id_x_x','sikka_sub_task_operation_x_x','office_x_x','guarantor_x','patient','description','visible_to_patient','primary_insurance_estimate','secondary_insurance_estimate','plan_sr_no','insurance_payment','approved','from_pms_x_x','id_y_x','from_pms_y_x','sikka_sub_task_y_x','sikka_sub_task_operation_y_x','uuid','guarantor_y','office_y_x','fullname','first_name','last_name','address','phone','zip_code','practice','primary_insurance_company_id','profile_picture','date_joined','zip_code_flag','address_comp','final_zip_code','id_x_y','from_pms_x_y','office_x_y','treatment','id_y_y','from_pms_y_y','sikka_sub_task_y_y','sikka_sub_task_operation_y_y','office_y_y','procedure_code_x','procedure_code_description','patient_friendly_description','treatment_id','price','fetched_zip_code_flag','zip_code_verified','fetched_zip_code', 'entry_date', 'tooth_from']

df = drop_high_nan_columns(df) 
df = df.drop(columns = trash_columns) # Drop trash cols
df = merge_columns_with_priority(df,'health_category','procedure_code_category','treatment_category') # merge to get treatment_category with pattern from two category groups
df = df.dropna() # Drop the remaining NA
df, df_dict = object_processing(df) # Project columns dtype = object; For columns containing datetime, convert it to pd.datetime; For other object columns, encode them
df = days_cal(df,'procedure_date','birth_date', 'procedure_age') # age group preparation (modify to int)
df = days_cal(df,'last_visit','first_visit','visit_period')

# # Perform one-hot encoding on the "procedure_code_y" and "treatment_category"
df = one_hot_encode_procedure_and_treatment(df)
# df.to_csv(r'C:\Frank\UoT 2024-2025\MIE8888 Project\M18\output.csv', index=False)

df_next = df.copy() # for next step use

C:\Users\frank\AppData\Local\Temp\ipykernel_30592\2822141788.py:10: DtypeWarning: Columns (1,4,10,16,52,85,86,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_dataset, encoding="Windows-1252")
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-1\helper_functions.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_dt = pd.to_datetime(df[col], errors='raise')
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-1\helper_functions.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_dt = pd.to_datetime(df[col], errors='raise')
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-1\helper_functions.py:40: UserWarning: Could not infer format, so each element will

In [4]:
df_next.tail()

,provider_x,amount,treatment_plan_status,patient_id,city,state_y,sikka_provider_id,LATITUDE,LONGITUDE,procedure_age,...,treatment_category_12,treatment_category_13,treatment_category_14,treatment_category_15,treatment_category_16,treatment_category_17,treatment_category_18,treatment_category_19,treatment_category_20,treatment_category_21
354032,1,25.22,0,7997,15,0,0,43.670184,-79.517310,8.265753,...,False,False,False,False,False,False,False,False,False,False
354033,1,96.26,0,7997,15,0,0,43.670184,-79.517310,8.265753,...,False,False,False,False,False,False,False,False,False,False
354034,4,23.66,0,3107,0,0,0,43.801319,-79.564734,46.665753,...,False,False,False,False,False,False,False,False,False,False
354035,3,87.06,0,2679,0,0,0,43.852601,-79.561398,20.534247,...,False,False,False,False,False,False,False,False,False,False
354036,4,23.66,0,8157,0,0,0,43.776973,-79.594405,7.213699,...,False,False,False,False,False,False,False,False,False,False


## Divide patient-level treatment dataset into three groups (V1, V2, V3) based on the continuity and duration of their visit history
Definitions:
- First Visit Date: Start of treatment history
- Last Visit Date: End of treatment history
- Max Gap: Largest time gap between consecutive visits
- Duration: Total time span = Last Visit Date - First Visit Date

V1 (Very Consistent): Max gap between visits is less than 2 years (Doesn’t matter how long the total history is, as long as it's consistent)  
V2 (Inconsistent but Long History): Max gap is 2 years or more; Total duration is 7 years or more  
V3 (Inconsistent and Short History): Everything else  